# Interest Rate Prediction in Colombia

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px


In [2]:
filepath = 'data/interest_data_clean.xls'
df_dict = pd.read_excel(filepath, sheet_name=None)
print(df_dict.keys())
print(df_dict['policy_rate_t1'].head())
print(df_dict['policy_rate_actual'].head())

dict_keys(['policy_rate_t1', 'policy_rate_t3', 'policy_rate_t6', 'policy_rate_t9', 'policy_rate_t11', 'policy_rate_actual'])
       Fecha  t     Media  Desv. Estandar  Mediana
0 2014-07-01  1  0.043720        0.001688    0.045
1 2014-08-01  1  0.045376        0.001751    0.045
2 2014-09-01  1  0.045655        0.001113    0.045
3 2014-10-01  1  0.045187        0.000667    0.045
4 2014-11-01  1  0.045069        0.000417    0.045
  Fecha (dd/mm/aaaa)  Tasa de intervención de política monetaria (%)
0         2023-06-04                                           13.25
1         2023-06-03                                           13.25
2         2023-06-02                                           13.25
3         2023-06-01                                           13.25
4         2023-05-31                                           13.25


In [3]:
#creates a list that includes the dataframes of the projections, without the dataframe of the actual values
projection_periods_list = list(df_dict.keys())
projection_periods_list = projection_periods_list[:-1]
projection_periods_list

#creates a dictionary to rename columns for ease of use
column_rename_dict = {'Fecha':'projection_date',
                      't':'projected_months',
                      'Media':'mean',
                      'Desv. Estandar':'std',
                      'Mediana':'median'}

In [4]:
#changes the column names of the dataframe containing the actual interest rate values for ease of use
df_dict['policy_rate_actual'].rename(columns={'Fecha (dd/mm/aaaa)':'date', 'Tasa de intervención de política monetaria (%)':'actual_rate'}, inplace=True)

#sorts the actual values so the classification is done correctly
df_dict['policy_rate_actual'].sort_values('date', inplace=True)

#classifies the actual values by direction
col = 'actual_rate'
conditions = [ df_dict['policy_rate_actual'][col] > df_dict['policy_rate_actual'][col].rolling(31).mean() , df_dict['policy_rate_actual'][col] < df_dict['policy_rate_actual'][col].rolling(31).mean() ]
classifications = ['Increasing', 'Decreasing']

df_dict['policy_rate_actual']['actual_rate_direction'] = np.select(conditions, classifications, default='Neutral')

df_dict['policy_rate_actual'].head(15)

,date,actual_rate,actual_rate_direction
8920,1999-01-01,26.0,Neutral
8919,1999-01-02,26.0,Neutral
8918,1999-01-03,26.0,Neutral
8917,1999-01-04,26.0,Neutral
8916,1999-01-05,26.0,Neutral
8915,1999-01-06,26.0,Neutral
8914,1999-01-07,26.0,Neutral
8913,1999-01-08,26.0,Neutral
8912,1999-01-09,26.0,Neutral
8911,1999-01-10,26.0,Neutral


In [5]:
#defines function used to classify where actuals sit compared to the analyst projections
def classify_deviation(df, mean, lower_boundary, upper_boundary, actual_data):
    conditions = [ df[actual_data] > df[upper_boundary] , (df[upper_boundary] >= df[actual_data]) & (df[actual_data] >= df[lower_boundary]) ]
    classifications = ['Above Mean + 1 std', 'Within 1 std']
    return np.select(conditions, classifications, default='Below Mean - 1 std')


In [6]:
#make Dataframes ready for graphs
for period in projection_periods_list:
    #rename columns
    df_dict[period].rename(columns=column_rename_dict, inplace=True)
    #change numbers from decimal (0.043) to percentage (4.3)
    df_dict[period]['mean'] = df_dict[period]['mean'] * 100
    df_dict[period]['std'] = df_dict[period]['std'] * 100
    df_dict[period]['median'] = df_dict[period]['median'] * 100
    #create a range between -1 std and +1 std
    df_dict[period]['mean-1std'] = df_dict[period]['mean'] - df_dict[period]['std']
    df_dict[period]['mean+1std'] = df_dict[period]['mean'] + df_dict[period]['std']
    #merge with the actual value using the date being projected
    df_dict[period]['projected_date'] = df_dict[period]['projection_date'] + pd.DateOffset(months=df_dict[period]['projected_months'].mean())
    df_dict[period] = df_dict[period].merge(df_dict['policy_rate_actual'], left_on='projected_date', right_on='date', how='left')
    df_dict[period].drop(columns='date', inplace=True)
    #calculate other fields necessary for graphs
    df_dict[period]['classification'] = classify_deviation(df_dict[period], 'mean', 'mean-1std', 'mean+1std', 'actual_rate')
    df_dict[period]['mean_v_median'] = np.absolute(df_dict[period]['mean'] - df_dict[period]['median'])
    df_dict[period]['median_accuracy'] = np.where(df_dict[period]['median'] == df_dict[period]['actual_rate'], 1, 0)

df_dict['policy_rate_t1'].head(15)

,projection_date,projected_months,mean,std,median,mean-1std,mean+1std,projected_date,actual_rate,actual_rate_direction,classification,mean_v_median,median_accuracy
0,2014-07-01,1,4.371951,0.168837,4.50,4.203114,4.540789,2014-08-01,4.25,Increasing,Within 1 std,0.128049,0
1,2014-08-01,1,4.537600,0.175080,4.50,4.362520,4.712680,2014-09-01,4.50,Increasing,Within 1 std,0.037600,1
2,2014-09-01,1,4.565476,0.111250,4.50,4.454226,4.676726,2014-10-01,4.50,Neutral,Within 1 std,0.065476,1
3,2014-10-01,1,4.518750,0.066687,4.50,4.452063,4.585437,2014-11-01,4.50,Neutral,Within 1 std,0.018750,1
4,2014-11-01,1,4.506944,0.041667,4.50,4.465278,4.548611,2014-12-01,4.50,Neutral,Within 1 std,0.006944,1
5,2014-12-01,1,4.506757,0.041100,4.50,4.465657,4.547857,2015-01-01,4.50,Neutral,Within 1 std,0.006757,1
6,2015-01-01,1,4.506098,0.039043,4.50,4.467054,4.545141,2015-02-01,4.50,Neutral,Within 1 std,0.006098,1
7,2015-02-01,1,4.494186,0.038125,4.50,4.456061,4.532311,2015-03-01,4.50,Neutral,Within 1 std,0.005814,1
8,2015-03-01,1,4.535714,0.130331,4.50,4.405384,4.666045,2015-04-01,4.50,Neutral,Within 1 std,0.035714,1
9,2015-04-01,1,4.511905,0.077152,4.50,4.434753,4.589056,2015-05-01,4.50,Neutral,Within 1 std,0.011905,1


In [7]:
#creates a dataframe with all the information in a friendlier format to use for visualizations in a dashboard
df_final = pd.concat((df_dict[period] for period in projection_periods_list), ignore_index=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   projection_date        535 non-null    datetime64[ns]
 1   projected_months       535 non-null    int64         
 2   mean                   535 non-null    float64       
 3   std                    535 non-null    float64       
 4   median                 535 non-null    float64       
 5   mean-1std              535 non-null    float64       
 6   mean+1std              535 non-null    float64       
 7   projected_date         535 non-null    datetime64[ns]
 8   actual_rate            505 non-null    float64       
 9   actual_rate_direction  505 non-null    object        
 10  classification         535 non-null    object        
 11  mean_v_median          535 non-null    float64       
 12  median_accuracy        535 non-null    int32         
dtypes: da

In [8]:
#exports the dataframe to a CSV file which will be used in a data visualization platform
df_final.to_csv('data/interest_dataframe_for_viz.csv', index=False)